In [ ]:
import tkinter as tk
import random

class Tetris:
    def __init__(self, master):
        self.master = master
        self.master.title("Tetris")
        self.width = 10
        self.height = 20
        self.cell_size = 30
        self.canvas = tk.Canvas(master, width=self.width*self.cell_size, height=self.height*self.cell_size, bg='black')
        self.canvas.pack()
        self.board = [[0 for _ in range(self.width)] for _ in range(self.height)]
        self.shapes = [
            [[1, 1, 1, 1]],  # I
            [[1, 1], [1, 1]],  # O
            [[0, 1, 0], [1, 1, 1]],  # T
            [[1, 0, 0], [1, 1, 1]],  # J
            [[0, 0, 1], [1, 1, 1]],  # L
            [[1, 1, 0], [0, 1, 1]],  # S
            [[0, 1, 1], [1, 1, 0]]   # Z
        ]
        self.colors = ['cyan', 'yellow', 'purple', 'blue', 'orange', 'green', 'red']
        self.reset_piece()
        self.game_over = False
        self.master.bind("<Key>", self.key_press)
        self.update()

    def reset_piece(self):
        self.shape_id = random.randint(0, len(self.shapes)-1)
        self.shape = self.shapes[self.shape_id]
        self.color = self.colors[self.shape_id]
        self.x = self.width // 2 - len(self.shape[0]) // 2
        self.y = 0

    def rotate(self):
        self.shape = [list(row) for row in zip(*self.shape[::-1])]
        if self.collide(self.x, self.y, self.shape):
            self.shape = [list(row) for row in zip(*self.shape)][::-1]  # undo

    def collide(self, x, y, shape):
        for i, row in enumerate(shape):
            for j, cell in enumerate(row):
                if cell:
                    if (x+j < 0 or x+j >= self.width or y+i >= self.height or
                        (y+i >= 0 and self.board[y+i][x+j])):
                        return True
        return False

    def freeze(self):
        for i, row in enumerate(self.shape):
            for j, cell in enumerate(row):
                if cell and self.y+i >= 0:
                    self.board[self.y+i][self.x+j] = self.color
        self.clear_lines()
        self.reset_piece()
        if self.collide(self.x, self.y, self.shape):
            self.game_over = True

    def clear_lines(self):
        new_board = [row for row in self.board if any(cell == 0 for cell in row)]
        lines_cleared = self.height - len(new_board)
        for _ in range(lines_cleared):
            new_board.insert(0, [0 for _ in range(self.width)])
        self.board = new_board

    def move(self, dx, dy):
        if not self.collide(self.x+dx, self.y+dy, self.shape):
            self.x += dx
            self.y += dy
        elif dy == 1:
            self.freeze()

    def key_press(self, event):
        if self.game_over:
            return
        if event.keysym == 'Left':
            self.move(-1, 0)
        elif event.keysym == 'Right':
            self.move(1, 0)
        elif event.keysym == 'Down':
            self.move(0, 1)
        elif event.keysym == 'Up':
            self.rotate()
        elif event.keysym == 'space':
            while not self.collide(self.x, self.y+1, self.shape):
                self.y += 1
            self.freeze()
        self.draw()

    def update(self):
        if not self.game_over:
            self.move(0, 1)
            self.draw()
            self.master.after(400, self.update)
        else:
            self.canvas.create_text(self.width*self.cell_size//2, self.height*self.cell_size//2,
                                    text="GAME OVER", fill="white", font=("Arial", 24))

    def draw(self):
        self.canvas.delete("all")
        # Draw board
        for y in range(self.height):
            for x in range(self.width):
                color = self.board[y][x]
                if color:
                    self.draw_cell(x, y, color)
        # Draw current piece
        for i, row in enumerate(self.shape):
            for j, cell in enumerate(row):
                if cell and self.y+i >= 0:
                    self.draw_cell(self.x+j, self.y+i, self.color)

    def draw_cell(self, x, y, color):
        x0 = x * self.cell_size
        y0 = y * self.cell_size
        x1 = x0 + self.cell_size
        y1 = y0 + self.cell_size
        self.canvas.create_rectangle(x0, y0, x1, y1, fill=color, outline='gray')

if __name__ == "__main__":
    root = tk.Tk()
    game = Tetris(root)
    root.mainloop()

In [31]:
import tkinter as tk
import math

class Kaleidoscope:
    def __init__(self, master, width=400, height=400, sectors=8):
        self.master = master
        self.width = width
        self.height = height
        self.sectors = sectors
        self.radius = min(width, height) // 2 - 10
        self.center = (width // 2, height // 2)
        self.canvas = tk.Canvas(master, width=width, height=height, bg='black')
        self.canvas.pack()
        self.last = None
        self.canvas.bind("<B1-Motion>", self.draw)
        self.canvas.bind("<ButtonRelease-1>", self.reset)

    def draw(self, event):
        if self.last is None:
            self.last = (event.x, event.y)
            return
        x0, y0 = self.last
        x1, y1 = event.x, event.y
        for i in range(self.sectors):
            angle = 2 * math.pi * i / self.sectors
            dx0 = x0 - self.center[0]
            dy0 = y0 - self.center[1]
            dx1 = x1 - self.center[0]
            dy1 = y1 - self.center[1]
            r0 = math.hypot(dx0, dy0)
            r1 = math.hypot(dx1, dy1)
            theta0 = math.atan2(dy0, dx0) + angle
            theta1 = math.atan2(dy1, dx1) + angle
            px0 = self.center[0] + r0 * math.cos(theta0)
            py0 = self.center[1] + r0 * math.sin(theta0)
            px1 = self.center[0] + r1 * math.cos(theta1)
            py1 = self.center[1] + r1 * math.sin(theta1)
            color = "#%02x%02x%02x" % (int(128+127*math.cos(angle)), int(128+127*math.sin(angle)), 200)
            self.canvas.create_line(px0, py0, px1, py1, fill=color, width=2)
        self.last = (x1, y1)

    def reset(self, event):
        self.last = None

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Kaleidoscope")
    kaleido = Kaleidoscope(root)
    root.mainloop()